In [1]:
import os, json
import pandas as pd
import numpy as np
import glob
import re
import tqdm as tqdm
import ast

In [2]:
from pathlib import Path
import pandas as pd

temp_df = pd.DataFrame()

path_to_json = './json_logs' 


json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)


dfs = [] # an empty list to store the data frames
for file in file_list:
    # print(str(file))
    data = pd.read_json(file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

temp_df = pd.concat(dfs ) #  (pd.Series)



In [3]:
result_column_df = temp_df['data']['result']
result_column_df

result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
result    [{'stream': {'app': 'worker-fetch-event', 'con...
Name: data, dtype: object

In [4]:
print(str(len(result_column_df)))

11


In [5]:

# Initialize an empty list to gather all 'values'
all_values = []
# Loop through the indices of temp_df
for i in range(len(result_column_df)):
    values = temp_df['data']['result'][i][0]['values']  # Access the 'values' field
    all_values.extend(values)  

# Convert the list of values into a DataFrame
values_df = pd.Series(all_values)

# Print the resulting DataFrame
# print(values_df)


/tmp/ipykernel_154225/3165085121.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  values = temp_df['data']['result'][i][0]['values']  # Access the 'values' field


In [6]:
print(str(len(values_df)))


# values_df.rename(columns={1:'Log String'}, inplace=True)


# print(values_df.columns)
values_df #.describe()
# all data found.


3508113


0          [1723106449759523051, [2024-08-08 15:40:49,759...
1          [1723106449749112639, [2024-08-08 15:40:49,748...
2          [1723106449748911046, 2024-08-08 15:40:49.748 ...
3          [1723106449748757857, 2024-08-08 15:40:49.748 ...
4          [1723106449748393296, 2024-08-08 15:40:49.747 ...
                                 ...                        
3508108    [1723914006968846879, 2024-08-18 00:00:06.968 ...
3508109    [1723914006968446207, 2024-08-18 00:00:06.968 ...
3508110    [1723914006967799471, 2024-08-18 00:00:06.967 ...
3508111    [1723914006967431632, 2024-08-18 00:00:06.966 ...
3508112    [1723914006964275493, [2024-08-18 00:00:06,963...
Length: 3508113, dtype: object

In [7]:
values_df_rev = values_df.iloc[::-1]

In [8]:
# The specific data point to count
search_string = '========================Start fetch event==========================='

# Count occurrences of the search string
count = sum('Start fetch event' in value[1] for value in values_df)
# Display the coun t
print(f"The data point appears {count} times.")

The data point appears 316067 times.


In [9]:
# The specific data point to count
search_string = 'Delay alarm event:' # August 10th alarm count.

# Count occurrences of the search string
count = sum(search_string in value[1] for value in values_df_rev)
# Display the coun t
print(f"The data point appears {count} times.")

print(str(len(values_df_rev)))

The data point appears 7433 times.
3508113


In [10]:
def collect_all_events(series):
    # Initialize lists to store indices
    start_indices = []
    end_indices = []
    
    # Iterate through the Series to find all start and end indices
    for i, value in tqdm.tqdm(enumerate(series), total=len(series), desc="Finding indices"):
        if 'Start fetch event' in str(value):
            start_indices.append(i)  # Capture the start index
        elif '======END======' in str(value):
            end_indices.append(i)  # Capture the end index

    # Gather all segments between each pair of start and end indices
    log_segments = []
    end_index = 0  # Initialize end index pointer

    for start in tqdm.tqdm(start_indices, desc="Gathering log segments"):
        # Move the end_index pointer to the first end index greater than start
        while end_index < len(end_indices) and end_indices[end_index] <= start:
            end_index += 1
        
        # If there is a valid end index, gather the log segment
        if end_index < len(end_indices):
            end = end_indices[end_index]
            segment = ' '.join(str(series[i]) for i in range(start + 1, end))
            log_segments.append(segment)

    # Create a DataFrame with an index column and the combined log string
    event_data_df = pd.DataFrame(log_segments, columns=['Log String'])
    event_data_df.index.name = 'Index'  # Set the index name

    return event_data_df

event_data_df = collect_all_events(values_df_rev) 


Gathering log segments: 100%|██████████| 316067/316067 [00:05<00:00, 56559.19it/s]


In [11]:
event_data_df

,Log String
Index,
0,"['1723106449748911046', '2024-08-08 15:40:49.7..."
1,"['1723106449691230012', '2024-08-08 15:40:49.6..."
2,"['1723106449624719370', '[2024-08-08 15:40:49,..."
3,"['1723106449572812804', '[2024-08-08 15:40:49,..."
4,"['1723106449506242161', '[2024-08-08 15:40:49,..."
...,...
316062,"['1723914016964469563', '[2024-08-18 00:00:16,..."
316063,"['1723914007264492991', '[2024-08-18 00:00:07,..."
316064,"['1723914007206810653', '[2024-08-18 00:00:07,..."


In [12]:
event_data_log_string = event_data_df['Log String']

print(event_data_log_string) # 34650 event, in a log string.


Index
0         ['1723106449748911046', '2024-08-08 15:40:49.7...
1         ['1723106449691230012', '2024-08-08 15:40:49.6...
2         ['1723106449624719370', '[2024-08-08 15:40:49,...
3         ['1723106449572812804', '[2024-08-08 15:40:49,...
4         ['1723106449506242161', '[2024-08-08 15:40:49,...
                                ...                        
316062    ['1723914016964469563', '[2024-08-18 00:00:16,...
316063    ['1723914007264492991', '[2024-08-18 00:00:07,...
316064    ['1723914007206810653', '[2024-08-18 00:00:07,...
316065    ['1723914007141020123', '[2024-08-18 00:00:07,...
316066    ['1723914007068239854', '2024-08-18 00:00:07.0...
Name: Log String, Length: 316067, dtype: object


In [13]:
# Function to extract args from log string
def get_args(log):
    if 'arg: {' in log:
        # Use a regular expression to find the arguments
        arg_match = re.search(r'arg:\s*(\{.*?\})', log)
        if arg_match:
            # Extract the matched group
            args_string = arg_match.group(1)
            
            # Convert the string representation of the dictionary to an actual dictionary
            args_dict = ast.literal_eval(args_string)
            # Normalize the dictionary into a DataFrame
            df = pd.json_normalize(args_dict)
            return df  # Return the DataFrame
        else:
            # Return a DataFrame with '0' values as strings if parsing fails
           return pd.DataFrame({'sn': ['Not Found'],
                                'user_id': ['Not Found'],
                                'token': ['Not Found'],
                                'time_millis': ['Not Found'], 
                                'encrypted_str': ['Not Found'], 
                                'time_query_latest': ['Not Found'], 
                                'datetime': ['Not Found']})
    else:
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        return pd.DataFrame({'sn': ['Not Found'],
                            'user_id': ['Not Found'],
                            'token': ['Not Found'], 
                            'time_millis': ['Not Found'],
                            'encrypted_str': ['Not Found'],
                            'time_query_latest': ['Not Found'], 
                            'datetime': ['Not Found']})

In [14]:
# 70s query time
def fetch_args():
    # List to store DataFrames
    args_dfs = []

    # Iterate over each log string and extract args
    for log in tqdm.tqdm((event_data_log_string), total=len(event_data_log_string), desc="Finding args"):
        ##### expect data error after enumerate.
        log = str(log)
        #print
        args_df = get_args(log)
        args_dfs.append(args_df)

    # Concatenate all DataFrames into a single DataFrame
    final_args_df = pd.concat(args_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_args_df)
    return(final_args_df)


fetch_args()

Finding args: 100%|██████████| 316067/316067 [00:49<00:00, 6447.04it/s]


                      sn                               user_id  \
0       ba39b3962a4bfe81  5afa5178-c4a6-4eda-af3d-4f816135cea4   
1              Not Found                             Not Found   
2              Not Found                             Not Found   
3              Not Found                             Not Found   
4              Not Found                             Not Found   
...                  ...                                   ...   
316062         Not Found                             Not Found   
316063         Not Found                             Not Found   
316064         Not Found                             Not Found   
316065  fc5f9a3b7707d489              65420522b8a6631f38d5e164   
316066         Not Found                             Not Found   

                                                    token  \
0       ZTMzOTRmOWRkY3xiYTM5YjM5NjJhNGJmZTgxfDVhZmE1MT...   
1                                               Not Found   
2                       

,sn,user_id,token,time_millis,encrypted_str,time_query_latest,datetime
0,ba39b3962a4bfe81,5afa5178-c4a6-4eda-af3d-4f816135cea4,ZTMzOTRmOWRkY3xiYTM5YjM5NjJhNGJmZTgxfDVhZmE1MT...,00002131723096002012,0dac76032657aaca2216eda2d6e4e0d7,2024-08-08 15:40:30,2024-08-08 15:40:39
1,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
2,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
3,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
4,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
...,...,...,...,...,...,...,...
316062,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
316063,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
316064,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found
316065,fc5f9a3b7707d489,65420522b8a6631f38d5e164,YWU4YTRkYTdjZXxmYzVmOWEzYjc3MDdkNDg5fDY1NDIwNT...,00001351723912232399,fc97940a8347f547ffd42247fb94bced,2024-08-17 23:59:47,2024-08-17 23:59:56


In [15]:

def get_request_event_list(log):
    if 'Request get event list' in log: # if string in log
        
        # Use a regular expression to find the arguments
        request_match = re.search(r'Request get event list: (.*?)}', log) # search string & end string 
        if request_match:
            # Extract the matched group
            request_string = request_match.group(1) + '}' # closing  signs
                # Convert the string representation of the dictionary to an actual dictionary
            request_dict = ast.literal_eval(request_string)
            # Normalize the dictionary into a DataFrame
            df = pd.json_normalize(request_dict)
            return df  # Return the DataFrame
            # except Exception as e:
        else:
            return pd.DataFrame({'sn': ['Not Found'],
                                'startTime': ['Not Found'],
                                'endTime': ['Not Found'],
                                })
    else:
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        return pd.DataFrame({'sn': ['Not Found'],
                    'startTime': ['Not Found'],
                    'endTime': ['Not Found'],
                    })



In [16]:
'''
fetch req event

'''

def fetch_request_event():

    # List to store DataFrames
    request_event_list_dfs = []
    # Iterate over each log string and extract args
    for log in event_data_log_string:
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  

        req_df = get_request_event_list(log)
        request_event_list_dfs.append(req_df) # append to extend. ()

    # Concatenate all DataFrames into a single DataFrame
    final_request_event_list_df = pd.concat(request_event_list_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_request_event_list_df)
    return(final_request_event_list_df)
fetch_request_event()



                      sn            startTime              endTime
0       ba39b3962a4bfe81  2024-08-08 15:40:30  2024-08-08 15:40:39
1              Not Found            Not Found            Not Found
2              Not Found            Not Found            Not Found
3              Not Found            Not Found            Not Found
4              Not Found            Not Found            Not Found
...                  ...                  ...                  ...
316062         Not Found            Not Found            Not Found
316063         Not Found            Not Found            Not Found
316064         Not Found            Not Found            Not Found
316065  fc5f9a3b7707d489  2024-08-17 23:59:47  2024-08-17 23:59:56
316066  fb1f6e5c62b71703  2024-08-17 23:59:47  2024-08-17 23:59:56

[316067 rows x 3 columns]


,sn,startTime,endTime
0,ba39b3962a4bfe81,2024-08-08 15:40:30,2024-08-08 15:40:39
1,Not Found,Not Found,Not Found
2,Not Found,Not Found,Not Found
3,Not Found,Not Found,Not Found
4,Not Found,Not Found,Not Found
...,...,...,...
316062,Not Found,Not Found,Not Found
316063,Not Found,Not Found,Not Found
316064,Not Found,Not Found,Not Found
316065,fc5f9a3b7707d489,2024-08-17 23:59:47,2024-08-17 23:59:56


In [17]:
'''   
get time query event
'''


def get_time_query_event(log):
    if 'Time query event: ' in log: # if string in log
        
        # Use a regular expression to find the arguments
        time_query_event_match = re.search(r'Time query event: (\d+\.\d{3})', log) # search string & end string 
        if time_query_event_match:
            # Extract the matched group
            time_query_event_string = time_query_event_match.group(1) # closing  signs
                # Convert the string representation of the dictionary to an actual dictionary
            # since its a raw string, cannot be turned to JSON dict.
            return pd.DataFrame({'TimeQueryEvent': [time_query_event_string]})
        else:
            return pd.DataFrame({'TimeQueryEvent': ['Not Found'],
                                })
    else:
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        return pd.DataFrame({'TimeQueryEvent': ['Not Found'],
                    })



In [18]:
''' Fetch time query event df
'''

def fetch_time_query_event():
        
    # List to store DataFrames
    time_query_event_dfs = []
    # Iterate over each log string and extract args
    for log in event_data_log_string:
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  # Assuming the first element is the log string

        time_query_event_df = get_time_query_event(log)
        time_query_event_dfs.append(time_query_event_df) # string error. 

    # Concatenate all DataFrames into a single DataFrame
    final_time_query_event_df = pd.concat(time_query_event_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_time_query_event_df)
    return(final_time_query_event_df)

fetch_time_query_event()

       TimeQueryEvent
0               0.038
1               0.036
2               0.033
3               0.045
4           Not Found
...               ...
316062      Not Found
316063          0.036
316064          0.037
316065          0.038
316066          0.096

[316067 rows x 1 columns]


,TimeQueryEvent
0,0.038
1,0.036
2,0.033
3,0.045
4,Not Found
...,...
316062,Not Found
316063,0.036
316064,0.037
316065,0.038


In [19]:
'''
function extract all time worker

'''

def get_all_time_worker(log):
    if 'All time worker' in log: # if string in log
        # Use a regular expression to find the arguments
        # regex search conflict.
        all_time_worker_match = re.search(r"All time worker: (.*?) s" , log)
        if all_time_worker_match:
            # Extract the JSON-like string
            all_time_worker_string = all_time_worker_match.group(1)  # Append closing braces
            # string type, do not need dict.
            df = pd.DataFrame({'AllTimeWorker': [all_time_worker_string]})
            return df
        else:  # not match   
            return pd.DataFrame({
                    'AllTimeWorker': ['Not Found'],
                    })    
    else:
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        return pd.DataFrame({
                    'AllTimeWorker': ['Not Found'],
                    })


In [20]:
def fetch_all_time_worker():

    # List to store DataFrames
    all_time_worker_dfs = []
    # Iterate over each log string and extract args
    for log in event_data_log_string:
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  # Assuming the first element is the log string

        all_time_worker_event_df = get_all_time_worker(log)
        all_time_worker_dfs.append(all_time_worker_event_df) # string error. 

    # Concatenate all DataFrames into a single DataFrame
    final_all_time_worker_df = pd.concat(all_time_worker_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_all_time_worker_df)
    return final_all_time_worker_df
    
fetch_all_time_worker()

               AllTimeWorker
0        0.04169297218322754
1        0.06423783302307129
2       0.037348031997680664
3        0.04944801330566406
4        0.04588580131530762
...                      ...
316062   0.03814101219177246
316063   0.04008007049560547
316064   0.04249310493469238
316065   0.04337191581726074
316066   0.10049295425415039

[316067 rows x 1 columns]


,AllTimeWorker
0,0.04169297218322754
1,0.06423783302307129
2,0.037348031997680664
3,0.04944801330566406
4,0.04588580131530762
...,...
316062,0.03814101219177246
316063,0.04008007049560547
316064,0.04249310493469238
316065,0.04337191581726074


In [21]:
'''  
function for delay alarm event (limited df.)

'''

def get_delay_alarm_event(log):
    if 'Delay alarm event: ' in log: # if string in log
        
        # Use a regular expression to find the arguments
        delay_alarm_event_match = re.search(r' Delay alarm event: (.*?) s', log) # search string & end string 
        if delay_alarm_event_match:
            # Extract the matched group
            delay_alarm_event_string = delay_alarm_event_match.group(1) # closing  signs
                # Convert the string representation of the dictionary to an actual dictionary
            # since its a raw string, cannot be turned to JSON dict.
            # print(delay_alarm_event_string)
            return pd.DataFrame({'DelayAlarm': [delay_alarm_event_string]})
        else:
            pass
            # return pd.DataFrame({'DelayAlarm': ['Not Found'],
            #                    })
    else:
        pass
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        # return pd.DataFrame({'DelayAlarm': ['Not Found'],
        #            })





In [22]:
def fetch_delay_alarm():
        # List to store DataFrames
    delay_alarm_dfs = []
    # Iterate over each log string and extract args
    for log in event_data_log_string:
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  # Assuming the first element is the log string

        delay_alarm_event_df = get_delay_alarm_event(log)
        delay_alarm_dfs.append(delay_alarm_event_df) # string error. 

    # Concatenate all DataFrames into a single DataFrame
    final_delay_alarm_df = pd.concat(delay_alarm_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_delay_alarm_df)
    return final_delay_alarm_df
    
fetch_delay_alarm()

     DelayAlarm
0            12
1            14
2            18
3            16
4            19
...         ...
4949         18
4950         13
4951         20
4952         11
4953         13

[4954 rows x 1 columns]


,DelayAlarm
0,12
1,14
2,18
3,16
4,19
...,...
4949,18
4950,13
4951,20
4952,11


In [23]:
'''   
time_process_home_face

'''

def get_time_process_home_face(log):
    if 'Time query event: ' in log: # if string in log
        
        # Use a regular expression to find the arguments
        time_time_process_home_face_match = re.search(r'Time query event: (\d+\.\d{3})', log) # search string & end string 
        if time_time_process_home_face_match:
            # Extract the matched group
            time_time_process_home_face_string = time_time_process_home_face_match.group(1) # closing  signs
                # Convert the string representation of the dictionary to an actual dictionary
            # since its a raw string, cannot be turned to JSON dict.
            return pd.DataFrame({'TimeHomeFace': [time_time_process_home_face_string]})
        else:
            return pd.DataFrame({'TimeHomeFace': ['Not Found'],
                                })
    else:
        # Return a DataFrame with '0' values as strings if 'arg: {' is not found
        return pd.DataFrame({'TimeHomeFace': ['Not Found'],
                    })

In [24]:
def fetch_time_process_home_face():
        # List to store DataFrames
    time_process_home_face_dfs = []
    # Iterate over each log string and extract args
    for log in event_data_log_string:
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  # Assuming the first element is the log string

        time_process_home_face_df = get_time_process_home_face(log)
        time_process_home_face_dfs.append(time_process_home_face_df) # string error. 

    # Concatenate all DataFrames into a single DataFrame
    final_time_process_home_face_df = pd.concat(time_process_home_face_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_time_process_home_face_df)
    return(final_time_process_home_face_df)
    
fetch_time_process_home_face()

       TimeHomeFace
0             0.038
1             0.036
2             0.033
3             0.045
4         Not Found
...             ...
316062    Not Found
316063        0.036
316064        0.037
316065        0.038
316066        0.096

[316067 rows x 1 columns]


,TimeHomeFace
0,0.038
1,0.036
2,0.033
3,0.045
4,Not Found
...,...
316062,Not Found
316063,0.036
316064,0.037
316065,0.038


In [25]:
df_result = pd.concat([ fetch_args(),
                        fetch_request_event(),
                        fetch_delay_alarm(),
                        fetch_time_query_event(),
                        fetch_time_process_home_face(),
                        fetch_all_time_worker(),
                        # final_response_event_df
                        ],
                        # ignore_index=True,
                        axis=1)
df_result
# linear runtime: 3mins

Finding args:   0%|          | 0/316067 [00:00<?, ?it/s]

Finding args: 100%|██████████| 316067/316067 [00:49<00:00, 6345.19it/s]


                      sn                               user_id  \
0       ba39b3962a4bfe81  5afa5178-c4a6-4eda-af3d-4f816135cea4   
1              Not Found                             Not Found   
2              Not Found                             Not Found   
3              Not Found                             Not Found   
4              Not Found                             Not Found   
...                  ...                                   ...   
316062         Not Found                             Not Found   
316063         Not Found                             Not Found   
316064         Not Found                             Not Found   
316065  fc5f9a3b7707d489              65420522b8a6631f38d5e164   
316066         Not Found                             Not Found   

                                                    token  \
0       ZTMzOTRmOWRkY3xiYTM5YjM5NjJhNGJmZTgxfDVhZmE1MT...   
1                                               Not Found   
2                       

,sn,user_id,token,time_millis,encrypted_str,time_query_latest,datetime,sn,startTime,endTime,DelayAlarm,TimeQueryEvent,TimeHomeFace,AllTimeWorker
0,ba39b3962a4bfe81,5afa5178-c4a6-4eda-af3d-4f816135cea4,ZTMzOTRmOWRkY3xiYTM5YjM5NjJhNGJmZTgxfDVhZmE1MT...,00002131723096002012,0dac76032657aaca2216eda2d6e4e0d7,2024-08-08 15:40:30,2024-08-08 15:40:39,ba39b3962a4bfe81,2024-08-08 15:40:30,2024-08-08 15:40:39,12,0.038,0.038,0.04169297218322754
1,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,14,0.036,0.036,0.06423783302307129
2,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,18,0.033,0.033,0.037348031997680664
3,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,16,0.045,0.045,0.04944801330566406
4,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,19,Not Found,Not Found,0.04588580131530762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316062,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,NaN,Not Found,Not Found,0.03814101219177246
316063,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,NaN,0.036,0.036,0.04008007049560547
316064,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,NaN,0.037,0.037,0.04249310493469238
316065,fc5f9a3b7707d489,65420522b8a6631f38d5e164,YWU4YTRkYTdjZXxmYzVmOWEzYjc3MDdkNDg5fDY1NDIwNT...,00001351723912232399,fc97940a8347f547ffd42247fb94bced,2024-08-17 23:59:47,2024-08-17 23:59:56,fc5f9a3b7707d489,2024-08-17 23:59:47,2024-08-17 23:59:56,NaN,0.038,0.038,0.04337191581726074


In [30]:
def get_response_alarm(log):
        if 'Response get event list: ' in log: 
                response_match = re.search(r" 'data': (.*?)'IsFinished': '1'}}" , log)
                if response_match:
                    response_string = response_match.group(1) + '}' 
                    response_string = response_string.replace("'", '"')
                    response_string = re.sub(r'"recface":\s*"({.*?})"', r'"recface": \1', response_string)
                    response_string = re.sub(r',\s*([\]})])', r'\1', response_string)
                    response_dict = ast.literal_eval(response_string)
                    df = pd.json_normalize(
                        response_dict, 
                        record_path=  ['AlarmArray'],
                        meta = ['SerialNumber','AlarmTotal']                    
                    )
                    return df
                else: 
                    return pd.DataFrame({
                            'SerialNumber': ['Not Found'],
                            'AlarmTotal': ['Not Found'],
                            
                            'AlarmEvent': ['Not Found'],
                            'AlarmId': ['Not Found'],
                            'AlarmMsg': ['Not Found'],
                            'AlarmTime': ['Not Found'],
                            'Channel': ['Not Found'],
                            
                            'PicInfo.ObjName': ['Not Found'],
                            'PicInfo.ObjSize': ['Not Found'],
                            'PicInfo.StorageBucket': ['Not Found'],
                            'VideoInfo.VideoLength': ['Not Found'],
                            'PicErr': ['Not Found'],
                            })    
        else:
            return pd.DataFrame({
                        'SerialNumber': ['Not Found'],
                        'AlarmTotal': ['Not Found'],
                        
                        'AlarmEvent': ['Not Found'],
                        'AlarmId': ['Not Found'],
                        'AlarmMsg': ['Not Found'],
                        'AlarmTime': ['Not Found'],
                        'Channel': ['Not Found'],
                        
                        'PicInfo.ObjName': ['Not Found'],
                        'PicInfo.ObjSize': ['Not Found'],
                        'PicInfo.StorageBucket': ['Not Found'],
                        'VideoInfo.VideoLength': ['Not Found'],
                        'PicErr': ['Not Found'],
                        })






In [31]:
def fetch_response():
    
    # List to store DataFrames
    response_event_dfs = []
    # Iterate over each log string and extract args
    for log in tqdm.tqdm((event_data_log_string), total=len(event_data_log_string), desc="Finding responses"):
        # Convert log string from list format to string if necessary
        if isinstance(log, list):
            log = log[0]  # Assuming the first element is the log string

        response_event_df = get_response_alarm(log)
        response_event_dfs.append(response_event_df) # string error. 

    # Concatenate all DataFrames into a single DataFrame
    final_response_event_df = pd.concat(response_event_dfs, ignore_index=True)

    # Display the final DataFrame
    print(final_response_event_df)
    return(final_response_event_df)


In [32]:
final_response_event_df = fetch_response()


Finding responses:   0%|          | 0/316067 [00:00<?, ?it/s]


TypeError: get_response_alarm() missing 1 required positional argument: 'log'

In [ ]:
final_response_event_df.to_csv('./csv_output/alarm_response_v3.csv', index=False)